In [ ]:
from sympy import *
from numpy import arange
%matplotlib inline

θ = symbols('theta',real=True)
erro = 0.0001
    
def GetElev(curva, h, β):
    if (curva == 'harmonica'):
        u = 0.5*h*( 1 - cos(pi*θ/β) )
    elif (curva == 'cicloide'):
        u = h*( θ/β - (1/(2*pi))*sin(2*pi*θ/β) )
    elif (curva == 'duplaharmonica'):
        u = 0.5*h*( 1 - cos(pi*θ/β) - 0.25*(1 - cos(2*pi*θ/β)) )
    elif (curva[0] in ['2','3','4']):
        curva = curva.replace('-',"")
        k = [int(i) for i in curva]
        if (len(k) == 2):
            a,b = k[0],k[1]
            u = h*(b*(θ/β)**a - a*(θ/β)**b)
        elif (len(k) == 3):
            a,b,c = k[0],k[1],k[2]
            u = 0.5*h*(b*c*(θ/β)**a - 2*a*c*(θ/β)**b + a*b*(θ/β)**c)
        elif (len(k) == 4):
            a,b,c,d = k[0],k[1],k[2],k[3]
            u = 0.5*h*( (1/3)*b*c*d*(θ/β)**a - a*c*d*(θ/β)**b + a*b*d*(θ/β)**c - (1/3)*a*b*c*(θ/β)**d )
    else:
        print('ERRO --> função '+curva+' inexistente')
        return 0
    return u

def bissect(f, L):
    f = Lambda( θ, f )
    
    a = L[0]; b = L[1]
    fa, fb = f(a), f(b)
    if ( fa*fb > 0 ):
        b += 0.1
    while abs(b-a) > 0.001:
        xm = (a+b)/2.0
        fm = f(xm)
        if fm*fa > 0.0: 
            a,fa = xm,fm
        else:
            b,fb = xm,fm
    return a,b

def Newton(f,x):     # Raiz de f por Newton Raphson
    df = diff(f, θ)
    f, df = Lambda( θ, f ), Lambda( θ, df )
    while ( abs( f(x) ) > 0.001 ):
        x = x - f(x)/(df(x))
    return x

def GetMaxPressureAng(curva, h, β, φ̂):
    u = GetElev(curva, h, β)
    v = diff(u,θ); a = diff(u,θ,2)
    
    uₜ = Lambda( θ, u )
    vₜ = Lambda( θ, v )

    ΔRₚ = 0.05
    Rₚ = 0.5*h
    T = tan(φ̂)
    Tφ = 1
    θₒ = 0.5*β
    L = [0.2,β]
    while ( Tφ - T > erro ):
        Rₚ += ΔRₚ
        L = bissect( a*(u + Rₚ)-v**2, L); θₒ = L[0]
        Tφ = vₜ(θₒ)/( uₜ(θₒ)+Rₚ )

    display(N(Rₚ,3))
    display(N(θₒ,3))
    plot( atan(v/(u+Rₚ)), (θ,0,β))

In [ ]:
GetMaxPressureAng('4-5-6-7', 10, pi/3, pi/6)